## Google BigData Showcase
BigQuery ML Tutorial 01 - Google Analytics로 구매 예측하기  
Edited by 김하제

### BigQuery 라이브러리 호출

In [1]:
from google.cloud import bigquery
client = bigquery.Client()

### 데이터셋 설정 "bqml_tutorial"

In [2]:
dataset = bigquery.Dataset(client.dataset('bqml_tutorial'))
dataset.location = 'US'

# Create Dataset
# client.create_dataset(dataset)

### Jupyter에서 BigQuery SQL을 사용하기 위한 magic 명령어 클라이언트 라이브러리 호출

In [3]:
%load_ext google.cloud.bigquery

### 1. Google Analytics Sessions 샘플 데이터 호출하기

In [4]:
%%bigquery
SELECT
    totals.transactions,
    device.operatingSystem,
    device.isMobile AS is_mobile,
    geoNetwork.country,
    totals.pageviews
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*`
WHERE _TABLE_SUFFIX BETWEEN '20160801' AND '20170630'
LIMIT 15

,transactions,operatingSystem,is_mobile,country,pageviews
0,None,Windows,False,Austria,1
1,None,Chrome OS,False,India,1
2,None,Macintosh,False,Taiwan,1
3,None,Windows,False,United Kingdom,1
4,None,Android,True,India,1
5,None,iOS,True,Serbia,1
6,None,Android,True,Ireland,1
7,None,Windows,False,France,1
8,None,Windows,False,United States,1
9,None,iOS,True,China,1


### 2. 모델 만들기

In [5]:
%%bigquery
CREATE OR REPLACE MODEL `bqml_tutorial.sample_model`
OPTIONS(model_type='logistic_reg') AS
SELECT
  IF(totals.transactions IS NULL, 0, 1) AS label,
  IFNULL(device.operatingSystem, "") AS os,
  device.isMobile AS is_mobile,
  IFNULL(geoNetwork.country, "") AS country,
  IFNULL(totals.pageviews, 0) AS pageviews
FROM
  `bigquery-public-data.google_analytics_sample.ga_sessions_*`
WHERE
  _TABLE_SUFFIX BETWEEN '20160801' AND '20170630'

""


### 3. 모델 학습 정보 보기

In [6]:
%%bigquery
SELECT
  *
FROM
  ML.TRAINING_INFO(MODEL `bqml_tutorial.sample_model`)

,training_run,iteration,loss,eval_loss,duration_ms,learning_rate
0,0,8,0.043878,0.045448,30886,25.6
1,0,7,0.044654,0.045502,29302,25.6
2,0,6,0.047345,0.048277,29521,12.8
3,0,5,0.053888,0.053340,30959,6.4
4,0,4,0.067776,0.066409,33883,3.2
5,0,3,0.097545,0.096206,28139,1.6
6,0,2,0.169802,0.168851,30954,0.8
7,0,1,0.320692,0.320175,30822,0.4
8,0,0,0.521573,0.521380,31146,0.2


### 4. 학습된 모델 평가하기

In [7]:
%%bigquery
SELECT
  *
FROM ML.EVALUATE(MODEL `bqml_tutorial.sample_model`, (
  SELECT
    IF(totals.transactions IS NULL, 0, 1) AS label,
    IFNULL(device.operatingSystem, "") AS os,
    device.isMobile AS is_mobile,
    IFNULL(geoNetwork.country, "") AS country,
    IFNULL(totals.pageviews, 0) AS pageviews
  FROM
    `bigquery-public-data.google_analytics_sample.ga_sessions_*`
  WHERE
    _TABLE_SUFFIX BETWEEN '20170701' AND '20170801'))

,precision,recall,accuracy,f1_score,log_loss,roc_auc
0,0.468504,0.110801,0.985343,0.179217,0.046242,0.982737


### 5. 평가된 모델로 예측하기

In [9]:
%%bigquery
SELECT
  country,
  SUM(predicted_label) as total_predicted_purchases
FROM ML.PREDICT(MODEL `bqml_tutorial.sample_model`, (
  SELECT
    IFNULL(device.operatingSystem, "") AS os,
    device.isMobile AS is_mobile,
    IFNULL(totals.pageviews, 0) AS pageviews,
    IFNULL(geoNetwork.country, "") AS country
  FROM
    `bigquery-public-data.google_analytics_sample.ga_sessions_*`
  WHERE
    _TABLE_SUFFIX BETWEEN '20170701' AND '20170801'))
  GROUP BY country
  ORDER BY total_predicted_purchases DESC
  LIMIT 10

,country,total_predicted_purchases
0,United States,220
1,Taiwan,8
2,Canada,7
3,Turkey,2
4,Japan,2
5,India,2
6,Vietnam,1
7,Guyana,1
8,Brazil,1
9,Germany,1


### 6. Test 데이터 생성하기

In [10]:
%%bigquery
SELECT
  fullVisitorId,
  SUM(predicted_label) as total_predicted_purchases
FROM ML.PREDICT(MODEL `bqml_tutorial.sample_model`, (
  SELECT
    IFNULL(device.operatingSystem, "") AS os,
    device.isMobile AS is_mobile,
    IFNULL(totals.pageviews, 0) AS pageviews,
    IFNULL(geoNetwork.country, "") AS country,
    fullVisitorId
  FROM
    `bigquery-public-data.google_analytics_sample.ga_sessions_*`
  WHERE
    _TABLE_SUFFIX BETWEEN '20170701' AND '20170801'))
  GROUP BY fullVisitorId
  ORDER BY total_predicted_purchases DESC
  LIMIT 10

,fullVisitorId,total_predicted_purchases
0,9417857471295131045,4
1,489038402765684003,2
2,0456807427403774085,2
3,2969418676126258798,2
4,112288330928895942,2
5,7420300501523012460,2
6,806992249032686650,2
7,2158257269735455737,2
8,057693500927581077,2
9,5073919761051630191,2
